In [1]:
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
STOPWORDS = thai_stopwords()
from glob import glob
from tqdm import tqdm
import json, html, re
import numpy as np
import pandas as pd
    
def clean(text, hashtag):
    text = text.replace(hashtag, '')
    text = html.unescape(text)
    text = re.sub(r'http.+?(?:\\s|$)', '', text) # URL link
    text = re.sub(r'[“”„]', '"', text) # convert double quotations into "
    text = re.sub(r'[‘’′′′′`]', "'", text) # convert single quotations into '
    text = re.sub(r'[ \u00a0\xa0\u3000\u2002-\u200a\t\n#]+', ' ', text) # shrink whitespaces e.g. good  boy -> good boy
    text = re.sub(r'[\r\u200b\ufeff]+', '', text) # remove non-breaking space
    text = re.sub(r'ํา','ำ', text) # am
    return text.strip()

def tokenize(text, hashtag):
    tokens = word_tokenize(clean(text, hashtag), keep_whitespace='False')
    tokens = [token for token in tokens if token not in STOPWORDS and re.match(r'[ก-๙][ก-๙\\.\\-]+$', token)]
    return tokens

In [2]:
jsons = glob('tweets/*.json')
for i, f in enumerate(jsons):
    print(i,f)

0 tweets/#กูสั่งให้มึงอยู่ใต้รัฐธรรมนูญ.json
1 tweets/#ธรรมศาสตร์และการชุมนุม.json
2 tweets/#เกียมอุดมไม่ก้มหัวให้เผด็จการ.json
3 tweets/#25พฤศจิกาไปSCB.json
4 tweets/#ม็อบ25พฤศจิกาทวงคืนสมบัติชาติ.json
5 tweets/#ม็อบ2ธันวา.json


In [15]:
%%time
hashtag = "#กูสั่งให้มึงอยู่ใต้รัฐธรรมนูญ"

df = pd.read_json(jsons[0], lines=True)
df['tokens'] = df.tweet.apply(lambda x: tokenize(x, hashtag))

df.tokens

CPU times: user 9.27 s, sys: 1.2 s, total: 10.5 s
Wall time: 12.1 s


0        [ผนงรจตกม, หัว, ขี่, เลื่อย, ปัญญา, บริหาร, ปร...
1        [อิสัส, ข้ออ้าง, รัฐประหาร, แบบนี้, จัญไร, ดีก...
2        [รัฐ, ทำ, ประชาชน, แบบนี้, สมควร, ไอ้, สลิ่ม, ...
3        [รด, เยี่ยว, สิ, โควิด, โควิด, ชักว่าว, ข่าว, ...
4            [ขุด, ลึก, เหี้ย, ข้าราชการ, ไทย, บ่อ, บาดาล]
                               ...                        
19060    [อี, แปล, ปฏิรูป, แฟน, มีต, อี, ไพร่, แปล, ปฏิ...
19061      [ไอ้, มือปราบ, หมู, กะทะ, เรือเป็ด, กุ้ง, ย่าง]
19062                                                   []
19063                                         [แท๊ก, รุ้ง]
19064                          [พรุ่งนี้, สภา, ร่าง, รธน.]
Name: tokens, Length: 19065, dtype: object

In [16]:
df.tokens.apply(len).mean()

8.542669813794912

# LDA

In [17]:
%%time

from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary

corpus_dictionary = Dictionary(df.tokens)
corpus = []
for tokens_list in tqdm(df.tokens):
    corpus.append(corpus_dictionary.doc2bow(tokens_list))
lda = LdaModel(corpus, num_topics=5, id2word=corpus_dictionary, passes=10)

100%|██████████| 19065/19065 [00:00<00:00, 73441.08it/s]


CPU times: user 1min 8s, sys: 525 ms, total: 1min 9s
Wall time: 1min 19s


In [18]:
print("|rank|topic 1||topic 2||topic 3||topic 4||topic 5||\n|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|")
for i in range(10):
    print(f'|{i+1}', end='')
    for j in range(5):
        word, score = lda.show_topic(j)[i]
        print(f"|{word}|{score:.3f}",end="")
    print('|')

|rank|topic 1||topic 2||topic 3||topic 4||topic 5||
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|1|ม็อบ|0.060|เหี้ย|0.052|ม็อบ|0.224|ม็อบ|0.087|ม็อบ|0.079|
|2|พฤศจิกา|0.052|คน|0.031|พฤศจิกา|0.215|พฤศจิกา|0.083|พฤศจิกา|0.077|
|3|น้ำ|0.037|ทำ|0.030|ประชาชน|0.068|ทางออก|0.076|นะคะ|0.024|
|4|ฉีด|0.029|อี|0.027|หยุด|0.026|ทุ|0.076|รี|0.016|
|5|กระสุน|0.024|หรอ|0.016|คุกคาม|0.019|กร่าง|0.076|อย่า|0.015|
|6|ยาง|0.022|ไอ้|0.016|เผด็จการ|0.019|สภา|0.057|ช่วยกัน|0.014|
|7|ชุมนุม|0.018|พ่อ|0.012|ตำรวจ|0.018|ประชุม|0.052|แบน|0.014|
|8|ใส่|0.015|รัก|0.011|ขี้ข้า|0.018|รัฐธรรมนูญ|0.048|ข่าว|0.011|
|9|แก๊สน้ำตา|0.013|อ่ะ|0.011|ทำร้าย|0.012|แก้|0.033|อมรินทร์|0.010|
|10|ตำรวจ|0.013|ประเทศ|0.011|พฤศจิกายน|0.008|ร่าง|0.007|ประชาธิปไตย|0.009|


# TF-IDF vectorization + SVD + K-means

In [19]:
%%time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans

### prepare data : list of sentences (delimitered by space) -> TF-IDF vectorization
corpus = df.tokens.apply(lambda x: ' '.join(x))
vectorizer = TfidfVectorizer(max_features=20000, ngram_range=(1,2))
X = vectorizer.fit_transform(corpus)
print(X.shape)

### singular value decomposition & normalize
dim = 50

svd = TruncatedSVD(n_components=dim, n_iter=7, random_state=42)
X = svd.fit_transform(X)
normalized = [vec/np.linalg.norm(vec) if np.linalg.norm(vec) != 0 else np.zeros(dim) for vec in X ]

(19065, 20000)
CPU times: user 3.22 s, sys: 340 ms, total: 3.56 s
Wall time: 2.37 s


In [21]:
%%time

### clustering 
num = 5

result = KMeans(n_clusters=num).fit_predict(normalized)

CPU times: user 2.41 s, sys: 516 ms, total: 2.93 s
Wall time: 878 ms


In [22]:
print("|tweet|topic|\n|:-:|:-:|")
for _ in range(30):
    i = np.random.randint(0, len(X))
    print(f"|{df.tweet[i]}|{result[i]+1}|")

|tweet|topic|
|:-:|:-:|
|ไอพวกเหี้ย ไอพวกเหี้ยยย โมโหมาก ได้เงินเท่าไหร่ห้ะ เค้าจ้างพวกมึงมาเท่าไหร่ #ม็อบ17พฤศจิกา #กูสั่งให้มึงอยู่ใต้รัฐธรรมนูญ|1|
|นี้สิ.. คนบนฟ้า​ #ประเทศกูมี #กูสั่งให้มึงอยู่ใต้รัฐธรรมนูญ|5|
|เอาอีกแล้ว@amarintvhd กล้าเรียกตัวเองว่าสื่อได้ไงอ่ะ ที่ด่าๆและแท็กไปคือไม่ได้ให้เซรีบรัมในหัวสมองได้ทำงานบ้างเลยหรอ #ม็อบ17พฤศจิกา  #กูสั่งให้มึงอยู่ใต้รัฐธรรมนูญ|5|
|#ม็อบ17พฤศจิกา #กูสั่งให้มึงอยู่ใต้รัฐธรรมนูญ #whatishappeninginthailand #รับทุกร่างคือทางออก #แบนอมรินทร์ #เบื่อม็อบมุ้งมิ้ง #แก้รัฐธรรมนูญ|3|
|คันปากมาก เห็นคลิปที่บางคนเอาน้ำไปแจกตำรวจ คืออะไร!!! น้ำแม่งยิ่งขาดแคลน กว่าจะส่งเข้าไปได้ หากันอย่างเร่งด่วน อิห่า ทำทำไม เลิกทำเป็นเรื่องโรแมนติกเถอะ จะอ้วก #ม็อบ17พฤศจิกา #สลายการชุมนุม #WhatsHappeningInThailand #กูสั่งให้มึงอยู่ใต้รัฐธรรมนูญ|5|
|สส โง่ @pprpthailand #ม็อบ17พฤศจิกา #กูสั่งให้มึงอยู่ใต้รัฐธรรมนูญ #ประชุมสภา|5|
|แบ่งพรรคแบ่งพวกชัดเจน... นี่หรือประณีประนอม!!?? #กูสั่งให้มึงอยู่ใต้รัฐธรรมนูญ #รับทุกร่างคือทางออก #กล้ามากเก่งมากขอบใจ #ต้องช่วยเอาความจร